In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os, glob, cv2, dlib, cv2
import numpy as np
import pandas as pd
from PIL import Image
import imutils
from imutils import face_utils
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

In [3]:
# Load dataset

data_train = '/content/drive/MyDrive/Researches/datasets/JAFFE_ID_Masked3/train/'
data_test  = '/content/drive/MyDrive/Researches/datasets/JAFFE_ID_Masked3/test/'

output_dir = '/content/drive/MyDrive/Researches/datasets/JAFFE_ID_Masked3/output/'

In [4]:
### Proses Listing Data Train
print('\n##### PROSES LISTING DATA TRAIN #####')
print('-------------------------------------')

labels_train = os.listdir(data_train)
labels_train.sort()

le = LabelEncoder()
le.fit(labels_train)

X_train_path = list()
y_train = list()

for label in labels_train:
    dir=os.path.join(data_train,label)
    n = 0
    for file in glob.glob(dir + "/*"):
        X_train_path.append(file)
        y_train.append(label)
        n+=1
    print('Listing data train pada label',label,'telah selesai. Jumlah data :', str(n))
    
y_train = le.transform(y_train)


##### PROSES LISTING DATA TRAIN #####
-------------------------------------
Listing data train pada label KA telah selesai. Jumlah data : 7
Listing data train pada label KL telah selesai. Jumlah data : 6
Listing data train pada label KM telah selesai. Jumlah data : 7
Listing data train pada label KR telah selesai. Jumlah data : 6
Listing data train pada label MK telah selesai. Jumlah data : 6
Listing data train pada label NA telah selesai. Jumlah data : 6
Listing data train pada label NM telah selesai. Jumlah data : 6
Listing data train pada label TM telah selesai. Jumlah data : 6
Listing data train pada label UY telah selesai. Jumlah data : 6
Listing data train pada label YM telah selesai. Jumlah data : 7


In [5]:
### Proses Listing Data Test
print('\n##### PROSES LISTING DATA TEST #####')
print('-------------------------------------')

labels_test = os.listdir(data_test)
labels_test.sort()

X_test_path = list()
y_test = list()

for label in labels_test:
    dir=os.path.join(data_test,label)
    n = 0
    for file in glob.glob(dir + "/*"):
        X_test_path.append(file)
        y_test.append(label)
        n += 1
    print('Listing data test pada label',label,'telah selesai. Jumlah data :', str(n))
    
y_test = le.transform(y_test)


##### PROSES LISTING DATA TEST #####
-------------------------------------
Listing data test pada label KA telah selesai. Jumlah data : 16
Listing data test pada label KL telah selesai. Jumlah data : 16
Listing data test pada label KM telah selesai. Jumlah data : 15
Listing data test pada label KR telah selesai. Jumlah data : 14
Listing data test pada label MK telah selesai. Jumlah data : 15
Listing data test pada label NA telah selesai. Jumlah data : 15
Listing data test pada label NM telah selesai. Jumlah data : 14
Listing data test pada label TM telah selesai. Jumlah data : 15
Listing data test pada label UY telah selesai. Jumlah data : 15
Listing data test pada label YM telah selesai. Jumlah data : 15


In [6]:
# Fungsi Extraksi Fitur
from skimage.feature import local_binary_pattern, hog
from datetime import datetime
import time

def save_image(img, prefix, dir):
  t = time.localtime()
  ct = time.strftime("%Y%m%d%H%M%S", t)
  filename = dir + prefix + ct + ".jpg"
  cv2.imwrite(filename, img)

def feature_lbph(image):
  if(len(image.shape)==3):
      image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  lbp = local_binary_pattern(image, 12, 2, method="uniform")
  n_bins = int(lbp.max() + 1)
  (hist, _) = np.histogram(lbp.ravel(),bins=n_bins,range=(0, n_bins))                                                 
  hist = hist.astype("float")                         
  hist /= (hist.sum() + 1e-7)
  fd = np.array(hist)
  return fd, lbp

def feature_hog(image):
  if(len(image.shape)==3):
      image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
  fd, img = hog(image, orientations=8, pixels_per_cell=(8, 8),
                  cells_per_block=(1, 1), visualize=True, multichannel=False)
  return fd, img


In [7]:
eyes_detector_path = "/content/drive/MyDrive/Researches/MFR_with_HOG_LBP_SVM/model/haarcascade_eye.xml"
eyes_detector = cv2.CascadeClassifier(eyes_detector_path)

In [12]:
import os

def prep_n_xtract(images, labels, mode):
  e = 0
  X_data = list()
  y_label = list()
  ft_lbp = list()
  ft_hog = list()

  for n, path in enumerate(images):
    # create output directory
    out = output_dir + mode + '/' + str(n) + '/'
    if not os.path.exists(out): os.makedirs(out)
    
    img = np.asarray(Image.open(path))
    # Save original image
    save_image(img,'0_ori_'+str(n)+'_', out)

    if len(img.shape)==2: img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.equalizeHist(gray)
    eyes = eyes_detector.detectMultiScale(
      gray, scaleFactor=1.05, minNeighbors=5,
      minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    if len(eyes)==2:
      [x1, y1, h1, w1], [x2, y2, h2, w2] = eyes
      if x1<x2:
        [lx, ly, lh, lw], [rx, ry, rh, rw] = eyes
      else:
        [rx, ry, rh, rw], [lx, ly, lh, lw] = eyes
      
      l_point = (int((lx+(lx+lw))/2), int((ly+(ly+lh))/2))
      r_point = (int((rx+(rx+rw))/2), int((ry+(ry+rh))/2))

      # Citra mata kanan kiri
      img_x = img.copy()
      img_x = cv2.circle(img_x, l_point, 2, (0, 0, 255), -1)
      img_x = cv2.circle(img_x, r_point, 2, (0, 0, 255), -1)
      save_image(img_x,'1_eye_'+str(n)+'_', out)

      # compute the angle between the eye centroids
      dY = r_point[1] - l_point[1]
      dX = r_point[0] - l_point[0]
      angle = np.arctan(dY/dX)
      angle = (angle * 180) / np.pi

      h, w = img.shape[:2]
      center = (w // 2, h // 2)
      M = cv2.getRotationMatrix2D(center, (angle), 1.0)
      img = cv2.warpAffine(img, M, (w, h))
      gray = cv2.warpAffine(gray, M, (w, h))

      img_x = cv2.warpAffine(img_x, M, (w, h))
      save_image(img_x,'2_rot_'+str(n)+'_', out)

      # Posisi titik mata setelah rotasi 
      new_points = []
      for point in [l_point, r_point]:
        # Convert to homogenous coordinates in np array format first so that you can pre-multiply M
        rotated_point = M.dot(np.array(point + (1,)))
        #img = cv2.circle(img, (int(rotated_point[0]), int(rotated_point[1])), 6, (0, 0, 255), -1)
        new_points.append(rotated_point)

      l_point, r_point = new_points
      
      # margin bawah = 1/2 * jarak mata
      jarak_mata = int(r_point[0]-l_point[0])
      margin = abs(int(jarak_mata/2))

      if l_point[0]>r_point[0]:
        x_min = int(r_point[0]-margin)
        y_min = int(r_point[1]-(2*margin))
        x_max = int(l_point[0]+margin)
        y_max = int(l_point[1]+margin)
      else:
        x_min = int(l_point[0]-margin)
        y_min = int(l_point[1]-(2*margin))
        x_max = int(r_point[0]+margin)
        y_max = int(r_point[1]+margin)    

      if x_min<0: x_min=0
      if y_min<0: y_min=0
      #if x_max>img.shape[0]: x_max = img.shape[0]
      #if y_max>img.shape[1]: y_max = img.shape[1]
      w = x_max - x_min
      h = y_max - y_min

      img_x = cv2.rectangle(img_x, (x_min, y_min), (x_max, y_max), (255, 0, 0), 2)
      save_image(img_x,"3_roibox_", out)

      img = img[y_min:y_min+h, x_min:x_min+w]
      img = imutils.resize(img, width=160)
      save_image(img,"4_roicrop_", out)
      if img.shape[0]>160: img = img[0:160, 0:160]

      # padding whitespace
      h = img.shape[0]
      w = img.shape[1]
      blank_image = np.zeros((160,160,3), np.uint8)
      blank_image[:,:] = (255,255,255)
      l_img = blank_image.copy()
      l_img[0:h, 0:160] = img.copy()
      img = l_img
      save_image(img,"5_roipad_", out)

      ttl = img.shape

      fea_lbph, img_lbp = feature_lbph(img)
      ft_lbp.append(fea_lbph)
      save_image(img_lbp, "6_lbp_", out)
      fea_hog, img_hog = feature_hog(img)
      ft_hog.append(fea_hog)
      save_image(img_hog, "7_hog_", out)

      global_feature = np.hstack([fea_lbph, fea_hog])
      X_data.append(global_feature)
      y_label.append(labels[n])
    else:
      e += 1
      ttl = "No eyes detected!"
      out = output_dir + '/error/' + mode + '/' + str(n) + '/'
      if not os.path.exists(out): os.makedirs(out)
      save_image(img, "err_", out)

  ft_lbp = np.asarray(ft_lbp)
  ft_hog = np.asarray(ft_hog)
  print(ft_lbp.shape, ft_hog.shape)
  X = np.asarray(X_data)
  y = np.asarray(y_label)

  return X, y  


In [13]:
X_train, y_train = prep_n_xtract(X_train_path, y_train, 'train')
print(X_train.shape, y_train.shape)

(51, 14) (51, 3200)
(51, 3214) (51,)


In [ ]:
X_test, y_test = prep_n_xtract(X_test_path, y_test, 'test')
print(X_test.shape, y_test.shape)

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, recall_score, log_loss, precision_score 
from sklearn.metrics import classification_report, confusion_matrix, roc_curve

clf = SVC(kernel='linear', probability=True)

clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)

train_predictions = clf.predict_proba(X_test)
acc= accuracy_score(y_test, y_predict)

print ('Accuracy:', acc)
print ('F1 score:', f1_score(y_test, y_predict, average='weighted', labels=np.unique(y_predict)))
print ('Recall:', recall_score(y_test, y_predict, average='weighted', labels=np.unique(y_predict)))
print ('Precision:', precision_score(y_test, y_predict, average='weighted', labels=np.unique(y_predict)))
#print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
print(classification_report(y_test, y_predict, target_names=labels_train))

In [ ]:
print(confusion_matrix(y_test, y_predict))

In [ ]:
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(clf, X_test, y_test, cmap='Blues', display_labels=labels_train) 
plt.show() 

In [ ]:
labels_train

In [ ]:
y_predict2 = clf.predict(X_train)
print(classification_report(y_train, y_predict2, target_names=labels_train))